In [32]:
!pip install bs4

In [33]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# library to handle requests
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# **1. Download and Explore Dataset**



In [34]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url)

In [35]:
My_table = soup.find('table',{'class':'wikitable sortable'})

 **Tranform the data into a *pandas* dataframe**

Get Table to Dataframe and Unassigned checks are made in Borough and Neighborhood

In [36]:
Head = My_table.find_all('tr')
Table = []
for th in Head:
    row = np.array(th.getText()[1:-1].split('\n'))
    if row[1] != 'Not assigned': 
        if row[2] == 'Not assigned':
            row[2] = row[1]
        Table.append(row)
        
df_Canada = pd.DataFrame(data=Table[1:], columns=Table[0])
df_Canada.head()

,Postal Code,,Borough,,Neighborhood
0,M1A,,,,Not assigned
1,M2A,,,,Not assigned
2,M3A,,North York,,Parkwoods
3,M4A,,North York,,Victoria Village
4,M5A,,Downtown Toronto,,"Regent Park, Harbourfront"


# **2. Trasnform Dataset**



In [37]:
Col = df_Canada.columns
Col

Index(['Postal Code', '', 'Borough', '', 'Neighborhood'], dtype='object')

Group the neighborhood by Postcode

In [38]:
Col = df_Canada.columns
df_Canada_Group = df_Canada.groupby(Col[0])[Col[4]].apply(lambda x: ''.join([str(elem+',') for elem in list(x)])).reset_index()
df_Canada_Group.head()

,Postal Code,Neighborhood
0,M1A,"Not assigned,"
1,M1B,"Malvern, Rouge,"
2,M1C,"Rouge Hill, Port Union, Highland Creek,"
3,M1E,"Guildwood, Morningside, West Hill,"
4,M1G,"Woburn,"


In [39]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighbourhood']
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended

In [40]:
neighborhoods

,Postal Code,Borough,Neighbourhood


In [41]:
df_repeated = df_Canada.sort_values('Postal Code', ascending=True)
df_repeated.drop_duplicates('Postal Code', keep="last", inplace=True)
df_Canada_Group['Borough'] = df_repeated.reset_index()['Borough']

In [42]:
df_Canada_Group

,Postal Code,Neighborhood,Borough
0,M1A,"Not assigned,",
1,M1B,"Malvern, Rouge,",Scarborough
2,M1C,"Rouge Hill, Port Union, Highland Creek,",Scarborough
3,M1E,"Guildwood, Morningside, West Hill,",Scarborough
4,M1G,"Woburn,",Scarborough
5,M1H,"Cedarbrae,",Scarborough
6,M1J,"Scarborough Village,",Scarborough
7,M1K,"Kennedy Park, Ionview, East Birchmount Park,",Scarborough
8,M1L,"Golden Mile, Clairlea, Oakridge,",Scarborough
9,M1M,"Cliffside, Cliffcrest, Scarborough Village West,",Scarborough


The dimensions of the dataset are observed

In [43]:
df_Canada_Group.shape


(180, 3)